In [1]:
# Importing the relevant libraries to complete the analysis
import numpy as np # linear algebra
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
Murex_access_report=pd.read_csv('Murex_access_formatted_report.csv')
Murex_access_report.head()

GPN  Access_order                  Segment_n_AccessRights
0  15914             1  DEP: Financial Controlling Services CC
1  15914             2  Middle Office - FRC (Read Only) : Read
2  27834             1              DEP: FRC Distribution EMEA
3  27834             2                     Sales - FRC : Write
4  31244             1       DEP: GALM Optimisation Costs (sg)

In [3]:
Murex_access_report = Murex_access_report.groupby(['GPN']).Segment_n_AccessRights.apply(np.array).reset_index()

In [4]:
Murex_access_report.head()

GPN                             Segment_n_AccessRights
0  15914  [DEP: Financial Controlling Services CC, Middl...
1  27834  [DEP: FRC Distribution EMEA, Sales - FRC : Write]
2  31244  [DEP: GALM Optimisation Costs (sg), Group Trea...
3  33710  [DEP: FRC Distribution EMEA, Sales - CREDIT : ...
4  50859  [DEP: GALM COO COSTS (SG), Offshore_Onshore - ...

In [5]:
# defining tranctionEncoder as te 
te = TransactionEncoder()

In [6]:
#transactions = Murex_access_report.iloc[:,1:].values
te_ary =te.fit(Murex_access_report['Segment_n_AccessRights']).transform(Murex_access_report['Segment_n_AccessRights'])

In [7]:
dataset = pd.DataFrame(te_ary, columns=te.columns_)
dataset.head()

BusMgmt - Rates : Read  Business Management - AUSTRALIA : Read  \
0                   False                                   False   
1                   False                                   False   
2                   False                                   False   
3                   False                                   False   
4                   False                                   False   

   Business Management - Credit : Read  Business Management - FRC : Read  \
0                                False                             False   
1                                False                             False   
2                                False                             False   
3                                False                             False   
4                                False                             False   

   Business Management - JAPAN : Read  Business Management - UK : Read  \
0                               False                            False   
1                               False                            False   
2                               False                            False   
3                               False                            False   
4                               False                            False   

   Business Management - US : Read  DEP: AMIT&-Core Operating Platform  \
0                            False                               False   
1                            False                               False   
2                            False                               False   
3                            False                               False   
4                            False                               False   

   DEP: APAC CH WM&CH IB Credit Risk Operation  DEP: BREXIT - Finance Support  \
0                                        False                          False   
1                                        False                          False   
2                                        False                          False   
3                                        False                          False   
4                                        False                          False   

   ...  Trading Swaps - USD - Trader : Write  \
0  ...                                 False   
1  ...                                 False   
2  ...                                 False   
3  ...                                 False   
4  ...                                 False   

   Trading Swaps_Bonds - CHF - Trader : Write  \
0                                       False   
1                                       False   
2                                       False   
3                                       False   
4                                       False   

   Trading Swaps_Bonds - EUR - Trader : Write  \
0                                       False   
1                                       False   
2                                       False   
3                                       False   
4                                       False   

   Trading Swaps_Bonds - GBP - Trader : Write  \
0                                       False   
1                                       False   
2                                       False   
3                                       False   
4                                       False   

   Trading UBS LTD - Trader : Write  Trading UBS LTD - Trader USD : Write  \
0                             False                                 False   
1                             False                                 False   
2                             False                                 False   
3                             False                                 False   
4                             False                                 False   

   Trading XCCY - Basis EMEA - Trader : Write  \
0                                       False   
1                  

In [8]:
frequent_itemsets = apriori(dataset, min_support=0.003, use_colnames=True)

In [ ]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules[ (rules['antecedent_len'] >=1)]

In [ ]:
# filtering guidance on https://www.ritchieng.com/pandas-multi-criteria-filtering/ see answer for how to filter with actor list
rules.loc[rules['antecedents'].str.contains('DEP'),:]

frequent_itemsets = apriori(dataset, min_support=0.00125, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules[ (rules['antecedent_len'] >= 1)]

In [ ]:
frequent_itemsets = apriori(dataset, min_support=0.003, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules[ (rules['antecedent_len'] >= 1) &
       (rules['lift'] > 1) ]

In [ ]:
rules.sort_values('confidence', ascending=False).head()

In [ ]:
# recalculate rules for entire dataset with lower support requirement
frequent_itemsets = apriori(dataset, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.4)
rules.head()

In [ ]:
# calculate average lift of entire dataset

# extact lift column from rules dataframe
# NOTE The column is wanted in series form to be passed to numpy. Extracting as a dataframe would be rules.loc[:, ["lift"]]
lift_series = rules.loc[:, "lift"]

# perform averaging with np.mean function
average_lift = np.mean(lift_series)

# NOTE this is an introduction into fstrings, current best practice for formatting strings
# additional resources can be found at: https://realpython.com/python-f-strings/
print(f"The average lift of the rules dataframe is: {average_lift}")

In [ ]:
# inspect the type of the 0th entry in the consequents column
print("The Type of this cell is:", type(rules.loc[0,"consequents"]))
# check the representation in Python
print("This appears as:", rules.loc[0,"consequents"])